In [1]:
import gc
import time
from logzero import logger
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
ls -lah ../data/

total 15G
drwxrwxr-x 5 ubuntu ubuntu 4.0K Apr 30 07:21 ./
drwxrwxr-x 7 ubuntu ubuntu 4.0K Apr 28 16:26 ../
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 16:52 .ipynb_checkpoints/
-rw-r--r-- 1 ubuntu ubuntu 4.8G Apr 30 14:27 click_data.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 29 14:21 kenkoooos/
-rw-r--r-- 1 ubuntu ubuntu 8.8G Apr 30 14:33 merge.feather
drwxrwxr-x 2 ubuntu ubuntu 4.0K Apr 28 17:13 raw/
-rw-rw-r-- 1 ubuntu ubuntu 1.4G Apr 28 17:20 raw.zip


In [3]:
%time
merge = pd.read_feather("../data/merge.feather", nthreads=4)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [4]:
train_size = merge[merge.is_test == 0].shape[0]

In [5]:
merge.shape

(203693876, 16)

In [6]:
merge.dtypes

app                      uint16
channel                  uint16
click_id                 uint32
device                   uint16
ip                       uint32
is_attributed             uint8
os                       uint16
click_time       datetime64[ns]
is_test                    bool
dow                       uint8
hour                      uint8
minute                    uint8
min5                      uint8
second                    uint8
sin_time                float64
cos_time                float64
dtype: object

### testの時刻分布が残念なので補正

In [7]:
%%time
train = merge[:train_size]
test = merge[train_size:]
del merge
gc.collect()

CPU times: user 40 ms, sys: 8 ms, total: 48 ms
Wall time: 44.5 ms


In [8]:
train.tail()

,app,channel,click_id,device,ip,is_attributed,os,click_time,is_test,dow,hour,minute,min5,second,sin_time,cos_time
184903402,12,340,99999999,1,121312,0,10,2017-11-09 16:00:00,False,3,16,0,0,0,-0.866025,-0.5
184903403,3,211,99999999,1,46894,0,19,2017-11-09 16:00:00,False,3,16,0,0,0,-0.866025,-0.5
184903404,1,274,99999999,1,320126,0,13,2017-11-09 16:00:00,False,3,16,0,0,0,-0.866025,-0.5
184903405,12,259,99999999,1,189286,0,37,2017-11-09 16:00:00,False,3,16,0,0,0,-0.866025,-0.5
184903406,11,137,99999999,1,106485,0,19,2017-11-09 16:00:00,False,3,16,0,0,0,-0.866025,-0.5


In [9]:
test.head()

,app,channel,click_id,device,ip,is_attributed,os,click_time,is_test,dow,hour,minute,min5,second,sin_time,cos_time
184903407,9,107,0,1,5744,99,3,2017-11-10 04:00:00,True,4,4,0,0,0,0.866025,0.5
184903408,9,466,1,1,119901,99,3,2017-11-10 04:00:00,True,4,4,0,0,0,0.866025,0.5
184903409,21,128,2,1,72287,99,19,2017-11-10 04:00:00,True,4,4,0,0,0,0.866025,0.5
184903410,15,111,3,1,78477,99,13,2017-11-10 04:00:00,True,4,4,0,0,0,0.866025,0.5
184903411,12,328,4,1,123080,99,13,2017-11-10 04:00:00,True,4,4,0,0,0,0.866025,0.5


In [10]:
test.groupby('hour').count()

,app,channel,click_id,device,ip,is_attributed,os,click_time,is_test,dow,minute,min5,second,sin_time,cos_time
hour,,,,,,,,,,,,,,,
4,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125
5,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427,2858427
6,381,381,381,381,381,381,381,381,381,381,381,381,381,381,381
9,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808
10,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993,3127993
11,413,413,413,413,413,413,413,413,413,413,413,413,413,413,413
13,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566
14,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257,3261257
15,499,499,499,499,499,499,499,499,499,499,499,499,499,499,499


In [11]:
print('hour==6:  ', test[test.hour == 6].click_time.min(), test[test.hour == 6].click_time.max())
print('hour==11: ', test[test.hour == 11].click_time.min(), test[test.hour == 11].click_time.max())
print('hour==15: ', test[test.hour == 15].click_time.min(), test[test.hour == 15].click_time.max())

hour==6:   2017-11-10 06:00:00 2017-11-10 06:00:00
hour==11:  2017-11-10 11:00:00 2017-11-10 11:00:00
hour==15:  2017-11-10 15:00:00 2017-11-10 15:00:00


In [12]:
%%time
test['click_time'] = (test.click_time.apply(lambda x: (x - pd.offsets.Second(1)) if x.hour in [6, 11, 15] else x))

CPU times: user 1min 3s, sys: 1.35 s, total: 1min 4s
Wall time: 1min 4s


In [13]:
test['hour'] = test.click_time.dt.hour.astype('uint8')
test['minute'] = test.click_time.dt.minute.astype('uint8')
test['min5'] = ((test.click_time.dt.minute/5).apply(lambda x: x//1)*5).astype('uint8')
test['second'] = test.click_time.dt.second.astype('uint8')

In [14]:
test.groupby('hour').count()

,app,channel,click_id,device,ip,is_attributed,os,click_time,is_test,dow,minute,min5,second,sin_time,cos_time
hour,,,,,,,,,,,,,,,
4,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125,3344125
5,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808,2858808
9,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808,2984808
10,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406,3128406
13,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566,3212566
14,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756,3261756


In [15]:
print(test.shape)
test.dtypes

(18790469, 16)


app                      uint16
channel                  uint16
click_id                 uint32
device                   uint16
ip                       uint32
is_attributed             uint8
os                       uint16
click_time       datetime64[ns]
is_test                    bool
dow                       uint8
hour                      uint8
minute                    uint8
min5                      uint8
second                    uint8
sin_time                float64
cos_time                float64
dtype: object

In [16]:
gc.collect()

31

### testに存在しない時間帯のデータを破棄

In [17]:
train = train[(train.hour >= 3) & (train.hour <= 15)]

In [18]:
print(train.shape)
train.groupby('hour').count()

(126923238, 16)


,app,channel,click_id,device,ip,is_attributed,os,click_time,is_test,dow,minute,min5,second,sin_time,cos_time
hour,,,,,,,,,,,,,,,
3,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476,9743476
4,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316,11223316
5,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359,10059359
6,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879,9471879
7,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535,9265535
8,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051,8582051
9,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837,9015837
10,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031,9982031
11,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918,9912918


### 統合して保存

In [19]:
%%time
merge = pd.concat([train, test])
del train, test
gc.collect()

CPU times: user 3.54 s, sys: 3.8 s, total: 7.35 s
Wall time: 7.35 s


In [20]:
%%time
merge.reset_index(inplace=True)
merge = merge.drop('index', axis=1)

CPU times: user 4.34 s, sys: 4.3 s, total: 8.64 s
Wall time: 8.63 s


In [21]:
merge.to_feather("../data/merge.feather")